In [1]:
import json
import pickle
import random
import math
from time import time
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, concatenate, Dense, Input, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [2]:
class ImageCaptionEmbeddingWithConsensus:
    def __init__(self, image_name, caption, image_embedding, caption_embedding, consensus):
        self.image_name = image_name
        self.caption = caption
        self.image_embedding = image_embedding
        self.caption_embedding = caption_embedding
        self.consensus = consensus

In [ ]:
image_caption_objects = 'corrected_images_captions_with_embeddings.pkl'
mirrored_objects = 'mirrored_images_captions_with_embeddings.pkl'
aug_data = 'augmented_data_embeddings.pkl'
aug_data_mirrored = 'augmented_data_embeddings_mirrored.pkl'

# encoding = latin1: needs to be specified in order to unpickle numpy arrays from Python2 to Python3
with open(image_caption_objects, 'rb') as f1:
    data_list_1 = pickle.load(f1, encoding="latin1")
    print(len(data_list_1))
    
with open(mirrored_objects, 'rb') as f2:
    data_list_2 = pickle.load(f2, encoding='latin1')
    print(len(data_list_2))
    
with open(aug_data, 'rb') as f3:
    data_list_3 = pickle.load(f3, encoding='latin1')
    print(len(data_list_3))
    
with open(aug_data_mirrored, 'rb') as f4:
    data_list_4 = pickle.load(f4, encoding='latin1')
    print(len(data_list_4))

In [ ]:
data_list = data_list_1 + data_list_2 + data_list_3 + data_list_4
print(len(data_list))

In [31]:
# This cell is here to run the model with human only ratings - Don't run this unless needed
# In these pkl files, the "consensus" field contains the human average of the human ratings
image_caption_objects = 'human_scores_only_embeddings.pkl'
mirrored_objects = 'human_scores_only_embeddings_mirrored.pkl'

with open(image_caption_objects, 'rb') as f1:
    data_list_1 = pickle.load(f1, encoding="latin1")
    print(len(data_list_1))
    
with open(mirrored_objects, 'rb') as f2:
    data_list_2 = pickle.load(f2, encoding='latin1')
    print(len(data_list_2))
    

data_list = data_list_1 + data_list_2
print(len(data_list))

12039
12039
24078


In [17]:
# This cell is here to run the model with vsepp only ratings - Don't run this unless needed
# In these pkl files, the "consensus" field contains the vsepp bucketed rating (as pulled from game data)
# Since we are not relying on human ratings here, we have in total 24,078 samples (12,039 original + mirrored), without aug
# With augmented data, we have 34,078
image_caption_objects = 'vsepp_scores_only_embeddings.pkl'
mirrored_objects = 'vsepp_scores_only_embeddings_mirrored.pkl'
aug_data = 'augmented_data_embeddings.pkl'
aug_data_mirrored = 'augmented_data_embeddings_mirrored.pkl'

with open(image_caption_objects, 'rb') as f1:
    data_list_1 = pickle.load(f1, encoding="latin1")
    print(len(data_list_1))
    
with open(mirrored_objects, 'rb') as f2:
    data_list_2 = pickle.load(f2, encoding='latin1')
    print(len(data_list_2))
  
with open(aug_data, 'rb') as f3:
    data_list_3 = pickle.load(f3, encoding='latin1')
    print(len(data_list_3))
    
with open(aug_data_mirrored, 'rb') as f4:
    data_list_4 = pickle.load(f4, encoding='latin1')
    print(len(data_list_4))

data_list = data_list_1 + data_list_2 + data_list_3 + data_list_4
print(len(data_list))

12039
12039
5000
5000
34078


In [18]:
print(data_list[1089].consensus)

3


In [19]:
# To concatenate, use numpy concatenate. axis = 1 results in a numpy array [[2048]], need to access [0]
im_emb = data_list[0].image_embedding
cap_emb = data_list[0].caption_embedding
concat = np.concatenate((im_emb, cap_emb), axis = 1)

In [20]:
# May need to exclude some data - especially data with no human ratings
# Note that it seems like with no human ratings, there is still consensus.
# And if there are no cosine similiarty data, there is still consensus ('Z0', 'Z1' don't always exist)
#list(filter(lambda x: x < 0, number_list))
cleaned_data_list = list(filter(lambda obj: float(obj.consensus) > 0, data_list))
random.shuffle(cleaned_data_list)
print(len(cleaned_data_list))

34078


In [21]:
im_cap_concat_list = [np.concatenate((obj.image_embedding, obj.caption_embedding), axis=1)[0].tolist() for obj in cleaned_data_list]
im_cap_concat = np.array(im_cap_concat_list)

# Note that the consensus is rounded, then subtract 1 so that the scores are 0 - 4
labels_list = [round(float(obj.consensus)) - 1 for obj in cleaned_data_list]
labels = np.array(labels_list)

In [22]:
print('Count 1 = ', labels_list.count(0))
print('Count 2 = ', labels_list.count(1))
print('Count 3 = ', labels_list.count(2))
print('Count 4 = ', labels_list.count(3))
print('Count 5 = ', labels_list.count(4))

Count 1 =  7950
Count 2 =  4236
Count 3 =  6892
Count 4 =  9174
Count 5 =  5826


In [23]:
print(len(im_cap_concat))
print(len(labels))
print(len(im_cap_concat[0]))
print(im_cap_concat.shape)
print(im_cap_concat[0].shape)
print(im_cap_concat[0])

34078
34078
2048
(34078, 2048)
(2048,)
[-0.030927   -0.01787462  0.01601018 ...  0.02482926  0.02433899
 -0.00618975]


In [24]:
early_stopping_monitor = EarlyStopping(patience=3)
batch_size = 100
num_epochs = 500
learning_rate = 0.00001
decay_rate = learning_rate / num_epochs

model_2 = Sequential()
model_2.add(Dense(512, activation='relu', input_shape=(2048,)))
model_2.add(Dropout(0.2))
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.2))
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.2))
#model_2.add(Dense(512, activation='relu'))
model_2.add(Dense(5, activation='softmax'))
opt = Adam(learning_rate=learning_rate, decay=decay_rate)
loss_func = SparseCategoricalCrossentropy()
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
# NOTE: to use categorical cross entropy, I had to subtract 1 from the consensus to make the ratings from 0 to 4
model_2.compile(optimizer=opt, loss=loss_func, metrics=['accuracy'])

In [25]:
train_size = math.floor(len(im_cap_concat) * 0.8)
print(train_size)

test_size = len(im_cap_concat) - train_size
print(test_size)

model_2.fit(im_cap_concat[:train_size], labels[:train_size], batch_size=batch_size, epochs=num_epochs, \
            validation_split=0.2, callbacks=[tensorboard])



27262
6816
Epoch 1/500
219/219 [==============================] - 1s 6ms/step - loss: 1.5900 - accuracy: 0.2632 - val_loss: 1.5747 - val_accuracy: 0.2727
Epoch 2/500
219/219 [==============================] - 1s 5ms/step - loss: 1.5697 - accuracy: 0.2686 - val_loss: 1.5596 - val_accuracy: 0.2727
Epoch 3/500
219/219 [==============================] - 1s 5ms/step - loss: 1.5548 - accuracy: 0.2723 - val_loss: 1.5397 - val_accuracy: 0.2809
Epoch 4/500
219/219 [==============================] - 1s 5ms/step - loss: 1.5307 - accuracy: 0.2951 - val_loss: 1.5069 - val_accuracy: 0.3064
Epoch 5/500
219/219 [==============================] - 1s 5ms/step - loss: 1.4951 - accuracy: 0.3275 - val_loss: 1.4674 - val_accuracy: 0.3457
Epoch 6/500
219/219 [==============================] - 1s 5ms/step - loss: 1.4590 - accuracy: 0.3480 - val_loss: 1.4335 - val_accuracy: 0.3567
Epoch 7/500
219/219 [==============================] - 1s 5ms/step - loss: 1.4280 - accuracy: 0.3629 - val_loss: 1.4089 - val_accur

In [26]:
predictions = model_2.predict(im_cap_concat[train_size:])

In [27]:
test_loss, test_accuracy = model_2.evaluate(im_cap_concat[train_size:], labels[train_size:])

213/213 [==============================] - 0s 2ms/step - loss: 1.1717 - accuracy: 0.7393


In [28]:
train_loss, train_accuracy = model_2.evaluate(im_cap_concat[:train_size], labels[:train_size])

852/852 [==============================] - 2s 2ms/step - loss: 0.2602 - accuracy: 0.9343


In [29]:
print(predictions[0])

predicted_ratings = []
for pred in predictions:
    predicted_ratings.append(np.argmax(pred)+1)

print(len(predicted_ratings))

# Need to add back the 1 that was subtracted for training
actual_ratings = [num + 1 for num in labels[train_size:]]
print(len(actual_ratings))

[1.1492788e-03 4.1326715e-04 7.9855919e-01 1.9684714e-01 3.0311155e-03]
6816
6816


In [30]:
def find_accuracy(predicted, actual):
    totals_by_rating = [0, 0, 0, 0, 0, 0]
    correct_by_rating = [0, 0, 0, 0, 0, 0]
    num_correct = 0
    num_ratings = 0
    
    for pred, act in zip(predicted, actual):
        totals_by_rating[act] += 1
        if pred == act:
            num_correct += 1
            correct_by_rating[act] += 1
        num_ratings += 1
        
    print('Accuracy = ', num_correct / num_ratings)
    
    print('---------------------------')
    print('Accuracy by rating 1 - 5')
    for idx, (correct, total) in enumerate(zip(correct_by_rating, totals_by_rating)):
        if idx == 0:
            continue
        print('Rating = ', idx, '| Accuracy = ', correct/total)
    
    
find_accuracy(predicted_ratings, actual_ratings)

Accuracy =  0.7392899061032864
---------------------------
Accuracy by rating 1 - 5
Rating =  1 | Accuracy =  0.8163265306122449
Rating =  2 | Accuracy =  0.7680652680652681
Rating =  3 | Accuracy =  0.665680473372781
Rating =  4 | Accuracy =  0.7161500815660685
Rating =  5 | Accuracy =  0.7330434782608696


In [ ]:
print(predicted_ratings)

Using keras tuner for hyperaparameter tuning

In [ ]:
#!pip install -U keras-tuner

In [ ]:
import kerastuner as kt

def build_model(hp):
    inputs = tf.keras.Input(shape=(2048,))
    x = inputs
    for _ in range(3):
        x = tf.keras.layers.Dense(512, activation='relu')(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout', 0, 0.3, step=0.05, default=0.2))(x)
    
    outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 0.00001, 0.00002, step=0.000005)),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
    return model
                
                
#tuner = kt.Hyperband(build_model, objective='val_accuracy', max_epochs=300, hyperband_iterations=2, directory='augmented_3layer')

tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=50, directory='random_search_sept_20')

tuner.search(im_cap_concat[:train_size], labels[:train_size], epochs=300, validation_split=0.2, callbacks=[EarlyStopping('val_loss', patience=3)])


In [ ]:
best_model = tuner.get_best_models(1)[0]

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print(best_hyperparameters.values)

In [ ]:
tuner.get_best_models()[0].summary()

In [ ]:
tuner.results_summary()